In [26]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/probando/review_2018.csv')
df['date'] = pd.to_datetime(df['date'])
df.info()
# Selecciona todas las columnas para las filas con índices entre 0 y 10
df = df.iloc[:100]
#We add a blank space before each review, because I didn't want to convert the first word of each review
df.text = ' '+df.text
#capitalize(): to change the first letter from a word to lowercase
#strip(): to remove empty spaces from the left and right
df.text = df.text.str.capitalize().str.strip()
#df['text'] = df['text'].astype('string')
# df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906362 entries, 0 to 906361
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    906362 non-null  object        
 1   user_id      906362 non-null  object        
 2   business_id  906362 non-null  object        
 3   stars        906362 non-null  int64         
 4   useful       906362 non-null  int64         
 5   funny        906362 non-null  int64         
 6   cool         906362 non-null  int64         
 7   text         906362 non-null  object        
 8   date         906362 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 62.2+ MB


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Tokenización

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('word_tokenize')
nltk.download('punkt')
# importing libraries
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import nltk.data
set(stopwords.words('english'))
# set of stop words
stop_words = set(stopwords.words('english')) 
# word_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
def not_stopwords(frase):   

# tokens of words  
  word_tokens = word_tokenize(frase) 

  filtered_sentence = [] 

  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 

  return " ".join(filtered_sentence) 

In [29]:
df.text = df.text.apply(lambda x: not_stopwords(x))
df.text.iloc[2]

"boyfriend tried deli first time today . turkey , avocado & bacon panini ha buffalo chicken wrap . definitely returning . wait food n't long , always appreciated lunch hour . much choose . salads , soup , macaroni , sandwiches hot food . love deli many options choose !"

In [30]:
# #filtrar caracteres especiales dentro de un dataframe con la libreria re en python
import re
df.text = df.text.apply(lambda x: re.sub("[^a-zA-Z]"," ",str(x)))
# now
print(df.text.loc[2])

boyfriend tried deli first time today   turkey   avocado   bacon panini ha buffalo chicken wrap   definitely returning   wait food n t long   always appreciated lunch hour   much choose   salads   soup   macaroni   sandwiches hot food   love deli many options choose  


Stemmer

In [31]:
df.text = df.text.apply(lambda x: word_tokenize(x))
df.text
from nltk.stem import PorterStemmer

def stemmering(phrase):
  Stem_words = []
  ps =PorterStemmer()
  for w in phrase:
      rootWord=ps.stem(w)
      Stem_words.append(rootWord)
  return Stem_words

In [32]:
df.text = df.text.apply(lambda x: stemmering(x))
df.text 

0     [decid, eat, awar, go, take, hour, begin, end,...
1     [realli, star, one, love, th, street, nake, tc...
2     [boyfriend, tri, deli, first, time, today, tur...
3     [amaz, biscuit, fill, blank, great, cocktail, ...
4     [cafe, extrem, cute, came, am, even, jazz, ban...
                            ...                        
95    [ve, time, n, t, disappoint, yet, meat, eater,...
96    [place, search, friend, tri, sever, nail, salo...
97    [uniqu, profession, base, experi, oper, utmost...
98    [top, notch, street, taco, great, servic, hand...
99    [holi, bat, crap, place, amaz, go, place, st, ...
Name: text, Length: 100, dtype: object

FreqDist: Cuantas veces se repite por review

In [33]:
import nltk
df.text = df.text.apply(lambda x: nltk.FreqDist(x))
df.text.iloc[0]

FreqDist({'take': 2, 'time': 2, 'experi': 2, 'long': 2, 'usual': 2, 'decid': 1, 'eat': 1, 'awar': 1, 'go': 1, 'hour': 1, ...})

In [34]:
#aqui lo convirtiio rapido prq era un dict_items
def one_word(frase):
  frase = dict(sorted(frase.items(), key=lambda item:item[1], reverse=True))
  frase = list(frase.items())[0]
  return frase[0]


In [35]:
one_word(df.text.iloc[0])

'take'

Palabra q mas ses repite por review en la columna 'text', faltaria filtrar un poco mas creo q sacare las palabras menos de 3 letras, y sustantivos eso tratare de hcaerlo con el notebook segundo

In [36]:
df.text = df.text.apply(lambda x: one_word(x))
df.text 

0          take
1         sushi
2          deli
3          amaz
4     breakfast
        ...    
95           ve
96        place
97         base
98          top
99        place
Name: text, Length: 100, dtype: object

In [38]:
df.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,take,2018-07-07 22:09:11
1,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,sushi,2018-07-17 03:30:07
2,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5,0,0,0,deli,2018-08-23 21:39:38
3,E9AB7V4z8xrt2uPF7T55FQ,iYY5Ii1LGpZCpXFkHlMefw,Zx7n8mdt8OzLRXVzolXNhQ,5,0,0,0,amaz,2018-04-27 23:03:21
4,A4n4YaE-owOVgTQcrVqHUw,S7bjj-L07JuRr-tpX1UZLw,I6L0Zxi5Ww0zEWSAVgngeQ,4,0,0,0,breakfast,2018-07-07 20:50:12
